<a href="https://colab.research.google.com/github/hubertwel/paragraph-similarity/blob/main/paragraph-similarity/paragraph_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import gensim
import os
import smart_open
import collections
import random

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        next(f) # skip a header row
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# Set file names for train and test data
# test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
test_data_dir = os.path.join('/content/drive/MyDrive/Colab Notebooks/data/', 'gouvfr', 'CorpusRandomTwitter')
print("test_data_dir: %s" % test_data_dir)
train_file = os.path.join(test_data_dir, 'randomtweets3.txt')
test_file = os.path.join(test_data_dir, 'randomtweets4.txt')

train_corpus = list(read_corpus(train_file))
test_corpus = list(read_corpus(test_file, tokens_only=True))
print('train_corpus the beginning: ', train_corpus[:2])
print(len(train_corpus))
print('test_corpus the beginning: ', test_corpus[:2])
print(len(test_corpus))
print()

# Build a vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)

# Train the model on the corpus
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
print()

# Assessing the model
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    # sanity check (self-similarity)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])

print('ranks: ', ranks)
print('second ranks: ', second_ranks)
counter = collections.Counter(ranks)
print(counter)
print()

# Testing the model
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
print('RANDOM DOCUMENT ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))

# Compare and print the most/second-most/third-most/median/least similar documents from the train corpus
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('THIRD-MOST', 2), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))


test_data_dir: /content/drive/MyDrive/Colab Notebooks/data/gouvfr/CorpusRandomTwitter
train_corpus the beginning:  [TaggedDocument(words=['rt', 'americanidol', 'watch', 'as', 'ddlovato', 'gives', 'an', 'amazing', 'performance', 'of', 'her', 'single', 'stonecold', 'demionidol'], tags=[0]), TaggedDocument(words=['https', 'co', 'hy', 'wzicz'], tags=[1])]
1209
test_corpus the beginning:  [['live', 'morning', 'news', 'weather', 'and', 'traffic', 'updates', 'fromâ', 'wdsu', 'https', 'co', 'vb', 'zq'], ['when', 'catch', 'fire', 'and', 'watch', 'over', 'you', 'like', 'the', 'sun', 'https', 'co', 'bi', 'rji']]
1213


ranks:  [7, 52, 110, 39, 72, 145, 1, 5, 10, 9, 0, 0, 0, 100, 21, 0, 33, 138, 0, 3, 8, 21, 102, 25, 237, 0, 51, 23, 39, 9, 28, 0, 0, 52, 4, 83, 202, 32, 105, 29, 91, 50, 50, 14, 237, 25, 1, 132, 21, 60, 102, 1110, 3, 125, 1168, 119, 0, 3, 124, 0, 110, 5, 75, 31, 1, 32, 8, 38, 61, 40, 218, 2, 647, 59, 0, 2, 16, 49, 31, 576, 14, 65, 0, 30, 3, 93, 8, 34, 194, 0, 4, 11, 139, 552, 50, 8,

In [ ]:
pip install --pre --upgrade gensim

     |████████████████████████████████| 24.0MB 1.4MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
